# Introduction
In this project we are going to analyze 'Brain tumor dataset', a T1 weighted MRI dataset from figshare,
<br>
apply preprocessing functions and create image classification model based on Convolutional neural network that predicts brain tumor type.
<br><br>
**Contents:**
1) **Data exploration**
    <br>
    1.1 - Image extraction <br>
    1.2 - Splitting Data <br>
    1.3 - Preview images <br>
    1.4 - Black/White balance <br>
    1.5 - Skull size <br>
    1.6 - Class weights <br>
    1.7 - Feature scaling <br>
2) **Neural Network Model**
    <br>
    2.1 - Building a model <br>
    2.2 - Visualising results <br>
    2.3 - Testing the model <br>
3) **Conclusion**
<br>

Highly reccomended to read the **data-description.txt** file to understand data organization.

# I Data exploration
#### 1.1 - Image extraction
The Dataset is in .mat format, so we extract the image values to regular image files.
<br>
For this task we're going to use h2py and cv2 libraries and iterate through mat dataset.
<br><br>
*We don't really need to do that, but I want to show the process as working with regular images.
<br>
Technically we supposed to to assign labels to integer values because later on we'll use loss='sparse_categorical_crossentropy' which is prepared for said value type.

In [78]:
import tensorflow as tf
import numpy as np
import os

# apply stable seed to avoid inconsistent results every time we test new setup
tf.config.experimental.enable_op_determinism()
tf.keras.utils.set_random_seed(0)
tf.random.set_seed(0)
np.random.seed(0)

# define paths for datasets
mat_path = 'D:\\data-science\\brain-tumor-data\\mat'
data_path = 'D:\\data-science\\brain-tumor-data\\images'
data_path_split = data_path + '_split'

mask_path = 'D:\\data-science\\brain-tumor-data\\mask'
mask_path_split = mask_path + '_split'

matrices = os.listdir(mat_path)

In [50]:
import matplotlib.pyplot as plt

# DON'T MIND THIS, it's dark theme for matplotlib
WHITE_MID = '#b5b5b5'
GREY_DARK = '#141414'

plt.rcParams['figure.facecolor'] = GREY_DARK
plt.rcParams['text.color'] = WHITE_MID
plt.rcParams['axes.facecolor'] = GREY_DARK
plt.rcParams['axes.edgecolor'] = WHITE_MID
plt.rcParams['axes.labelcolor'] = WHITE_MID
plt.rcParams['axes.titlecolor'] = WHITE_MID

plt.rcParams['grid.color'] = WHITE_MID
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['axes.grid'] = True

plt.rcParams['axes.linewidth'] = 1
plt.rcParams['xtick.color'] = WHITE_MID
plt.rcParams['ytick.color'] = WHITE_MID
plt.rcParams['legend.edgecolor'] = WHITE_MID
plt.rcParams['legend.labelcolor'] = WHITE_MID

In [134]:
from skimage import exposure
import cv2 as cv
import h5py

label_names = {1: 'meningioma',
               2: 'glioma',
               3: 'pituitary'}

# fig, axes = plt.subplots(2, 4, figsize=(20, 10))
# axes = axes.flatten()

# this iteration generates .png files in data_path directory
for i, mat in enumerate(matrices):
    # break
    
    # reads the .mat files from mat data path
    with h5py.File(f"{mat_path}\\{mat}", 'r') as file:
        image = file['cjdata']['image']
        label = int(file['cjdata']['label'][0])
        tumor_border = file['cjdata']['tumorBorder'][0]

        width, height = image.shape
        
        x_coords = [x for x in tumor_border[0::2]]
        y_coords = [y for y in tumor_border[1::2]]
        points = np.array([list(zip(y_coords, x_coords))], dtype=np.int32)
        
        binary_mask = np.zeros((width, height), dtype=np.uint8)
        cv.fillPoly(binary_mask, points, 1)
        
        # axes[i].set_axis_off()
        # axes[i].imshow(image, cmap='gray')
        # axes[i+4].set_axis_off()
        # axes[i+4].imshow(binary_mask, cmap='gray')

        # normalize values by maximum where [min, max] = [0, 1]
        image = np.array(image) / np.max(image)
        image *= 255
        image = np.array(image).astype(np.uint8)
        
        mask  = binary_mask * 255
        mask = np.array(mask).astype(np.uint8)
        
        # save the results to new data path

        cv.imwrite(os.path.join(mask_path, 'x-image', f"{str(i+1).zfill(4)}.png"), image)
        cv.imwrite(os.path.join(mask_path, 'y-mask',f"{str(i+1).zfill(4)}_mask.png"), mask)
print('Done!')

C:\Users\ati\AppData\Local\Temp\ipykernel_12364\929260822.py:19: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(file['cjdata']['label'][0])


Done!


#### 1.2 - Splitting data
For this task we use _splitfolders_ library. This makes us sure that data is correctly split without any leaks.
<br><br>
**Ratio:** <br>
train: 70% <br>
val: 20% <br>
test: 10%

In [150]:
import splitfolders

splitfolders.ratio(input=mask_path, output=mask_path_split, ratio=(.7, .2, .1), seed=0)

Copying files: 6128 files [00:05, 1205.63 files/s]


In [159]:
X_train = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'train', 'x-image'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
y_train = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'train', 'y-mask'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
train = tf.data.Dataset.zip((X_train, y_train))

X_val = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'val', 'x-image'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
y_val = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'val', 'y-mask'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
val = tf.data.Dataset.zip((X_val, y_val)) 

X_test = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'test', 'x-image'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
y_test = tf.keras.utils.image_dataset_from_directory(os.path.join(mask_path_split, 'test', 'y-mask'), batch_size=8, color_mode='grayscale', image_size=(256, 256), shuffle=False, labels=None)
test = tf.data.Dataset.zip((X_test, y_test)) 

Found 2144 files.
Found 2144 files.
Found 612 files.
Found 612 files.
Found 308 files.
Found 308 files.


In [165]:
data_iterator = train.as_numpy_iterator()
batch = data_iterator.next()

#### 1.3 - Preview the masks

In [166]:
import seaborn as sb

fig, axes = plt.subplots(nrows=2, ncols=8, figsize=(24, 6))
axes = axes.flatten()

images, masks = batch

for i in range(8):
    axes[i].imshow(images[i], cmap='gray')
    axes[i].set_xticklabels([])
    axes[i].set_yticklabels([])
    
    axes[i+8].imshow(masks[i], cmap='gray')
    axes[i+8].set_xticklabels([])
    axes[i+8].set_yticklabels([])

plt.tight_layout()
plt.show()

#### 1.5 Tumor size
By calculating amount of black pixels inside image we can extract images with specific skull/head size.
<br>
__*threshold*__ is max value that we consider/define as black background. The more the noisy our background is the higher the threshold we apply.
<br>
<br>
In summary this plot returns images where background fills specified % (see percentage variable) of area.
<br>


In [256]:
train_masks = np.concatenate([mask for mask in y_train])
test_masks = np.concatenate([mask for mask in y_test])
val_masks = np.concatenate([mask for mask in y_val])
all_masks = np.concatenate([train_masks, val_masks, test_masks])

dataset_white_pixels = []

for mask in all_masks:
    n_white_pixels = np.sum(mask == 255)
    percentage_white_pixels = (n_white_pixels / (256*256)) * 100
    dataset_white_pixels.append(percentage_white_pixels)

plt.figure(figsize=(16, 6))
sb.histplot(data=dataset_white_pixels, color=WHITE_MID, linewidth=1, edgecolor=GREY_DARK)
plt.title('Tumor size in pixels')
plt.xlabel('% of white pixels (tumor) filling mask image')
plt.tight_layout()

Most of tumors are small.

#### 1.6 - Class weights

In [37]:
files_per_label = lambda folder: os.listdir(os.path.join(data_path, folder))

# generate dictionary with lengths for each label
labels_len = {label: len(files_per_label(label)) for i, label in enumerate(labels)}

sb.barplot(data=labels_len, x=labels_len.keys(), y=labels_len.values(), color=WHITE_MID, linewidth=0)

print(f"{'Class':<10}   Count")
for label, count in labels_len.items():
    print(f"{label:<10} : {count}")

plt.title('Number of images')
plt.show()

Class        Count
glioma     : 1426
meningioma : 708
pituitary  : 930


**To handle class imbalance, we can calculate weights with given formula:**
<br><br>
$w_c = \frac{N}{N_c * C}$
<br><br>
where:
<br>
$ w_c $ = class weight <br>
$ N $ = number of samples in dataset <br>
$ N_c $ = number of samples in class <br>
$ C $ = number of unique classes in dataset

In [38]:
class_weights = dict()

for i, count in enumerate(labels_len.values()):
    class_weights[i] = sum(labels_len.values()) / (len(labels) * count)

print(f"{'Class':<10}   Weight")
for i, weight in class_weights.items():
    print(f"{labels[i]:<10} : {weight}")

Class        Weight
glioma     : 0.7162225338943432
meningioma : 1.4425612052730696
pituitary  : 1.0982078853046595


#### 1.7 - Feature scaling
Usually when we work with Images, the max value is 255 so we're going to transform our values from [min, max] = [0, 255] to [0, 1].

In [39]:
print(batch[0].min(), batch[0].max(), sep='\n')

0.0
250.75


In [40]:
train = train.map(lambda x, y: (x/255, y))
test = test.map(lambda x, y: (x/255, y))
val = val.map(lambda x, y: (x/255, y))

iterator = test.as_numpy_iterator()
batch = iterator.next()

min_value = iterator.next()[0].min()
max_value = iterator.next()[0].max()

print(min_value, max_value, sep='\n')

0.0
0.98137254


# II Neural Network Model
#### 2.1 - Building a model
In short our model base on convolutional layers, where each layer contain filter that convolve with input matrix (image) resulting filtered output.
<br>
As input shape we apply values that corresponds to:
- width = 256
- height = 256
- channel = 1 (grayscale).
<br>

__*ReLU*__ activation function cleans our output from all unnecessary negative values, replacing them with zeros.
<br>
__*Dropout*__ layers drops random neurons each time convolution layer is being applied, reducing chance to overfit.
<br>
Since our dataset contain 3 classes I applied __*softmax*__ activation to the last step.

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, ReLU
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [93]:
def create_model(num_filters=16, kernel_size=3, learning_rate=0, regulate=0, slope=0):
    
    model = Sequential()
    
    model.add(Input(shape=(256, 256, 1)))
    
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), 1, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(num_filters * 2, (kernel_size, kernel_size), 1, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    
    model.add(Conv2D(num_filters, (kernel_size, kernel_size), 1, activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling2D())
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])
    
    return model

In [94]:
model = create_model(learning_rate=0.0005)
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_30 (Conv2D)              │ (None, 254, 254, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 254, 254, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 127, 127, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 125, 125, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 125, 125, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 60, 60, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 60, 60, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 30, 30, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 14400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 256)            │     3,686,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,696,851 (14.10 MB)

 Trainable params: 3,696,851 (14.10 MB)

 Non-trainable params: 0 (0.00 B)

In [95]:
n_epochs = 20

early_stopping = EarlyStopping(
    monitor='val_loss',
    restore_best_weights=True,
    baseline=None,
    min_delta=0,
    patience=n_epochs,
    verbose=0,
)

In [96]:
hist = model.fit(train, validation_data=val, epochs=n_epochs, class_weight=class_weights, verbose=2, callbacks=[early_stopping])

Epoch 1/20
268/268 - 27s - 102ms/step - loss: 0.6682 - sparse_categorical_accuracy: 0.6637 - val_loss: 0.6406 - val_sparse_categorical_accuracy: 0.7337
Epoch 2/20
268/268 - 25s - 92ms/step - loss: 0.4556 - sparse_categorical_accuracy: 0.8018 - val_loss: 0.5746 - val_sparse_categorical_accuracy: 0.7304
Epoch 3/20
268/268 - 25s - 92ms/step - loss: 0.3292 - sparse_categorical_accuracy: 0.8554 - val_loss: 0.4751 - val_sparse_categorical_accuracy: 0.7712
Epoch 4/20
268/268 - 25s - 91ms/step - loss: 0.2172 - sparse_categorical_accuracy: 0.9086 - val_loss: 0.4267 - val_sparse_categorical_accuracy: 0.7941
Epoch 5/20
268/268 - 25s - 93ms/step - loss: 0.1726 - sparse_categorical_accuracy: 0.9296 - val_loss: 0.3975 - val_sparse_categorical_accuracy: 0.8072
Epoch 6/20
268/268 - 28s - 106ms/step - loss: 0.1231 - sparse_categorical_accuracy: 0.9515 - val_loss: 0.3416 - val_sparse_categorical_accuracy: 0.8366
Epoch 7/20
268/268 - 25s - 93ms/step - loss: 0.0915 - sparse_categorical_accuracy: 0.9669 - 

#### 2.2 - Visualising results

In [121]:
acc_labels = list(hist.history.keys())

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 10))
axes = axes.flatten()

sb.lineplot(data=hist.history, y='sparse_categorical_accuracy', x=[x for x in range(1, n_epochs+1, 1)], label='train', ax=axes[0], color=WHITE_MID)
sb.lineplot(data=hist.history, y='val_sparse_categorical_accuracy', x=[x for x in range(1, n_epochs+1, 1)], label='validation', ax=axes[0], linestyle='--', color=WHITE_MID)

sb.lineplot(data=hist.history, y='loss', x=[x for x in range(1, n_epochs+1, 1)], label='train', ax=axes[1], color=WHITE_MID)
sb.lineplot(data=hist.history, y='val_loss', x=[x for x in range(1, n_epochs+1, 1)], label='validation', ax=axes[1], linestyle='--', color=WHITE_MID)

sb.lineplot(data=hist.history, y='sparse_categorical_accuracy', x=[x for x in range(1, n_epochs+1, 1)], label='accuracy', ax=axes[2], color=WHITE_MID)
sb.lineplot(data=hist.history, y='loss', x=[x for x in range(1, n_epochs+1, 1)], label='loss', ax=axes[2], linestyle='--', color=WHITE_MID)

sb.lineplot(data=hist.history, y='val_sparse_categorical_accuracy', x=[x for x in range(1, n_epochs+1, 1)], label='accuracy', ax=axes[3], color=WHITE_MID)
sb.lineplot(data=hist.history, y='val_loss', x=[x for x in range(1, n_epochs+1, 1)], label='loss', ax=axes[3], linestyle='--', color=WHITE_MID)

for i in range(4):
    axes[i].legend(loc='lower right', bbox_to_anchor=(1.005, 1), ncol=2)
    axes[i].set_xlabel('Epoch')
    axes[i].set_ylim([0, 1])
    axes[i].set_xlim([1, n_epochs])
    axes[i].set_ylabel('')

axes[0].set_title('Accuracy (train / val)', weight='bold')
axes[1].set_title('Loss (train / val)', weight='bold')
axes[2].set_title('Train (accuracy / loss)', weight='bold')
axes[3].set_title('Validation (accuracy / loss)', weight='bold')

final_metrics = model.evaluate(val, verbose=0)
for key, value in zip(['val loss', 'val accuracy'], final_metrics):
    print(f"{key:>14}: {value}")

plt.tight_layout()

      val loss: 0.1830584704875946
  val accuracy: 0.9411764740943909


#### 2.3 - Testing the model
Both Training and Validation curves looks very promising, however we have to test our model for final results.

In [98]:
y_true = []
y_pred = []

for batch_images, batch_labels in test:
    y_true_batch = batch_labels
    y_pred_batch = np.argmax(model.predict(batch_images, verbose=0), axis=1)

    y_true.extend(y_true_batch)
    y_pred.extend(y_pred_batch)
    
y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [120]:
from sklearn.metrics import classification_report, confusion_matrix

report = classification_report(y_true, y_pred, target_names=labels)
print(report)

cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
fig = plt.figure(figsize=(4, 4))
fig = sb.heatmap(cm, annot=True, fmt='d', cbar=False, cmap='Greys_r', linewidths=0)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.axis('off')
plt.tight_layout()

              precision    recall  f1-score   support

      glioma       0.97      0.98      0.97       143
  meningioma       0.95      0.86      0.91        72
   pituitary       0.94      0.99      0.96        93

    accuracy                           0.95       308
   macro avg       0.95      0.94      0.95       308
weighted avg       0.95      0.95      0.95       308



# III Conclusion
We ended up with decent F1 score of 95%. Despite this fact, model still have small problem with classifying _meningioma_ class, propably because of it's size.
<br>
Note that the dataset itself is very good quality, it's balanced in terms of specific factors like: resolution, color range, object placement.
<br><br>
#### Significant improvement noticed after:
- Tweaking learning rate of the Adam optimizer
- Weighting classes
- Adding dropout layers for each convolution layer
<br>
instead of holding one dropout layer before output densing layers.
<br><br>
#### Ideas to improve:
- Preprocessing images with kmeans color clustering, didn't returned better results.
- Removing small images (head size), <br>
this meant to remove images where the number of black pixels exceeded a certain %.

#### Thanks for reading, I hope you enjoyed the analysis :] <br>
_Gracjan Pawłowski, 2024_